In [ ]:
import os
import glob
import pickle
import ast
import json
import numpy as np
import seaborn as sns
import pandas as pd
from dance_evaluation import *
import matplotlib.pyplot as plt

# Load genre mappings
with open("genreID_count_mapping.json", "r") as file:
    genre_Tcount = json.load(file)

with open("genre_symbols_mapping.json", "r") as file:
    genre_name = json.load(file)

In [ ]:
a = 60; b= 140
mode = "zero_uni"
metric = "pos"
w_sec = 5; 
h_sec = w_sec/2

read_dir = f"./saved_result/tempo_{a}_{b}"
fname1 = f"score/score_one_pos_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"
fname2 = f"score/score_one_vel_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"
fname3 = f"score/score_multi_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"

fpath1 = os.path.join(read_dir, fname1)
fpath2 = os.path.join(read_dir, fname2)
fpath3 = os.path.join(read_dir, fname3)

score_one_pos = pd.read_pickle(fpath1)
score_one_vel = pd.read_pickle(fpath2)
score_multi = pd.read_pickle(fpath3)

fpath4 = os.path.join(read_dir, f"pos/left_hand_y_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl")    # For reference only
main_df = pd.read_pickle(fpath4)
total = 1341

score = {"score_one_pos": score_one_pos, "score_one_vel": score_one_vel, "score_multi": score_multi}
# ['both_hand_x_both_foot_x', 'both_hand_y_both_foot_y', 
# 'both_hand_resultant_both_foot_resultant', 'bothhand_xy_bothfoot_xy']

#### For Score Multi

In [ ]:
bpm_dict = {"bpm_avg": 0,  "bpm_mode": 1, "bpm_median": 2}

scr_key = "score_multi"
mtyp = "posvel"
# seg_name = "both_hand_y_both_foot_y"
save_path1 = f"./saved_result/tempo_{a}_{b}/hit_genrewise_{scr_key}_{mtyp}_{mode}_{a}_{b}.pkl"

json_hits_genrewise = {}
for key in bpm_dict.keys():
    
    hits_list = {

        "both_hand_x_both_foot_x": score[scr_key][key]["both_hand_x_both_foot_x"][f'metrics_{mtyp}']['hits_idx'],
        'both_hand_y_both_foot_y': score[scr_key][key]['both_hand_y_both_foot_y'][f'metrics_{mtyp}']['hits_idx'],
        'bothhand_xy_bothfoot_xy': score[scr_key][key]['bothhand_xy_bothfoot_xy'][f'metrics_{mtyp}']['hits_idx'],
        'both_hand_resultant_both_foot_resultant': score[scr_key][key]['both_hand_resultant_both_foot_resultant'][f'metrics_{mtyp}']['hits_idx'],

        }

    # Initialize final dataframe with unique genres
    final_df = pd.DataFrame(list(genre_name.items()), columns=["dance_genre", "genre"])
    final_df["total"] = final_df["dance_genre"].map(genre_Tcount)

    # Dictionary to store genre-wise hit indices per limb
    genre_wise_hits = {limb: {} for limb in hits_list.keys()}

    # For each body segment, find genre-wise hit indices
    for limb, hit_indices in hits_list.items():
        hit_idx_list = list(hit_indices)
        hit_df = main_df.iloc[hit_idx_list]                    # DataFrame for estimated hits of that limb

        # Group by genre and store indices per genre
        for genre_id, group in hit_df.groupby("dance_genre"):
            genre_wise_hits[limb][genre_id] = set(group.index)      # Store as set for easy intersection

        # groups the DataFrame hit_df by the column "dance_genre",counts how many rows belong to each genre, 
        # and creates a new column to store these counts.
        grouped = hit_df.groupby(['dance_genre']).size().reset_index(name=f'{limb}_count')
        final_df = final_df.merge(grouped, on='dance_genre', how='left')
        final_df[f'{limb}_count'] = final_df[f'{limb}_count'].fillna(0).astype(int)
        final_df[f'{limb}_percentage'] = round((final_df[f'{limb}_count'] / final_df['total']) * 100, 2)

    # Clean final_df to keep genre and counts
    hits_genrewise_df = final_df[['genre', 'total'] + 
                                [f'{limb}_count' for limb in hits_list.keys()] + 
                                [f'{limb}_percentage' for limb in hits_list.keys()]]
    
    json_hits_genrewise[key] = hits_genrewise_df
    

with open(save_path1, "wb") as f:
    pickle.dump(json_hits_genrewise, f)

#### For Score One

In [ ]:
segment_keys = [
            "left_hand_x", "right_hand_x", "left_hand_y", "right_hand_y",   # singular
            "left_foot_x", "right_foot_x", "left_foot_y", "right_foot_y",   # singular
            
            "lefthand_xy", "righthand_xy", "leftfoot_xy", "rightfoot_xy",   # singular | 35, 40, 34, 36
            "left_hand_resultant", "right_hand_resultant", "left_foot_resultant", "right_foot_resultant",   # singular | 18,20,17,17 %
            
            "both_hand_x", "both_hand_y", "both_foot_x", "both_foot_y",
            "bothhand_xy", "bothfoot_xy", "both_hand_resultant", "both_foot_resultant", # resultant of x and y onsets
            ]

In [ ]:
bpm_dict = {"bpm_avg": 0,  "bpm_mode": 1, "bpm_median": 2}

mtyp = "vel"
scr_key = f"score_one_{mtyp}"

save_path1 = f"./saved_result/tempo_{a}_{b}/hit_genrewise_{scr_key}_{mode}_{a}_{b}.pkl"

json_hits_genrewise = {}
for key in bpm_dict.keys():
    
    hits_list = {
        seg: score[scr_key][key][seg][f'metrics_{mtyp}']['hits_idx']
        for seg in segment_keys
        }

    # Initialize final dataframe with unique genres
    final_df = pd.DataFrame(list(genre_name.items()), columns=["dance_genre", "genre"])
    final_df["total"] = final_df["dance_genre"].map(genre_Tcount)

    # Dictionary to store genre-wise hit indices per limb
    genre_wise_hits = {limb: {} for limb in hits_list.keys()}

    # For each body segment, find genre-wise hit indices
    for limb, hit_indices in hits_list.items():
        hit_idx_list = list(hit_indices)
        hit_df = main_df.iloc[hit_idx_list]                    # DataFrame for estimated hits of that limb

        # Group by genre and store indices per genre
        for genre_id, group in hit_df.groupby("dance_genre"):
            genre_wise_hits[limb][genre_id] = set(group.index)      # Store as set for easy intersection

        # groups the DataFrame hit_df by the column "dance_genre",counts how many rows belong to each genre, 
        # and creates a new column to store these counts.
        grouped = hit_df.groupby(['dance_genre']).size().reset_index(name=f'{limb}_count')
        final_df = final_df.merge(grouped, on='dance_genre', how='left')
        final_df[f'{limb}_count'] = final_df[f'{limb}_count'].fillna(0).astype(int)
        final_df[f'{limb}_percentage'] = round((final_df[f'{limb}_count'] / final_df['total']) * 100, 2)

    # Clean final_df to keep genre and counts
    hits_genrewise_df = final_df[['genre', 'total'] + 
                                [f'{limb}_count' for limb in hits_list.keys()] + 
                                [f'{limb}_percentage' for limb in hits_list.keys()]]
    
    json_hits_genrewise[key] = hits_genrewise_df
    

with open(save_path1, "wb") as f:
    pickle.dump(json_hits_genrewise, f)